import roboticstoolbox as rtb
robot = rtb.models.Panda()

In [ ]:
from roboticstoolbox.models.DH import Panda
panda = Panda()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
q_arr = np.random.rand(10,7)

In [ ]:
_path = panda.fkine_all(q_arr[0])

In [ ]:
_aa = _path[7]
_aa

In [ ]:
_aa.t

In [ ]:
def tray_nodes(q_arr, tool_l = 0.103):
    from roboticstoolbox.models.DH import Panda
    panda = Panda()
    panda.tool.t[2] = tool_l
    
    l = q_arr.shape[0]
    result = np.zeros([8,l,3])
    for ii in range(l):
        config = panda.fkine_all(q_arr[ii,:])
        for jj in range(8):
            result[jj,ii,:] = config[jj].t
    return result

In [ ]:
def end_eff(q_arr, tool_l = 0.103):
    from roboticstoolbox.models.DH import Panda
    panda = Panda()
    panda.tool.t[2] = tool_l
    
    l = q_arr.shape[0]
    result = np.zeros([l,3])
    for ii in range(l):
        config = panda.fkine(q_arr[ii,:])
        result[ii,:] = config.t
    return result

In [ ]:
tray_nodes(q_arr)

In [ ]:
panda.plot(q_arr[0])

In [ ]:
%matplotlib notebook

In [ ]:
q_tr = np.loadtxt('panda_q_traj.txt')

In [ ]:
q_tr.shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig_0 = plt.figure(figsize=[8,8])


ax = Axes3D(fig_0, 
            auto_add_to_figure=False,
            azim=0,
            elev=10,)
fig_0.add_axes(ax)

In [ ]:

_plot = panda.plot(q_tr[0], dt = 0.04, limits=[-0.5, 0.5, -0.5, 0.5, 0, 1], fig = fig_0,
           movie='panda_traj_0.gif',block = False)
_plot.ax.azim = -82


In [ ]:
panda.plot(q_tr[:5], dt = 0.04, limits=[-0.5, 0.5, -0.5, 0.5, 0, 1], fig = fig_0,
           movie='panda_traj_0.gif',block = False)

In [ ]:
_plot.ax.azim = -82

In [ ]:
_plot.step()

In [ ]:
panda.tool.t[2] = 0.103

In [ ]:
panda.tool.t[2] = 0.2

In [ ]:
panda.fkine(q0)

In [ ]:
panda.fkine_all(q0)

In [ ]:
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as geo
import meshcat.transformations as tf
import meshcat
import pinocchio as pin

In [ ]:
from example_robot_data import load

In [ ]:
robot = load('panda')

In [ ]:
robot.initViewer(loadModel=True)

In [ ]:
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

In [ ]:
# Start a new MeshCat server and client.
# Note: the server can also be started separately using the "meshcat-server" command in a terminal:
# this enables the server to remain active after the current script ends.
#
# Option open=True pens the visualizer.
# Note: the visualizer can also be opened seperately by visiting the provided URL.
try:
    viz.initViewer(open=False)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)

In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
viz.loadViewerModel()

In [ ]:
q0 = pin.neutral(robot.model)
viz.display(q0)

In [ ]:
q0[-2:] = [0.03,0.03]
viz.display(q0)
q0

In [ ]:
# Create a list of joints to lock
jointsToLock = ['panda_finger_joint1','panda_finger_joint2']
 
# Get the ID of all existing joints
jointsToLockIDs = []
for jn in jointsToLock:
    if robot.model.existJointName(jn):
        jointsToLockIDs.append(robot.model.getJointId(jn))
    else:
        print('Warning: joint ' + str(jn) + ' does not belong to the model!')

In [ ]:
robot = robot.buildReducedRobot(list_of_joints_to_lock=jointsToLockIDs, reference_configuration=q0)

In [ ]:
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

In [ ]:
# Start a new MeshCat server and client.
# Note: the server can also be started separately using the "meshcat-server" command in a terminal:
# this enables the server to remain active after the current script ends.
#
# Option open=True pens the visualizer.
# Note: the visualizer can also be opened seperately by visiting the provided URL.
try:
    viz.initViewer(open=False)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)

In [ ]:
viz.viewer.jupyter_cell()

display_robot_and_ball(q_tr, xyz_eff, frame_t = 0.02, t_end=1, n_extend = 40, brake_n = 15)

In [ ]:
viz.loadViewerModel()

In [ ]:
q0 = pin.neutral(robot.model)
viz.display(q0)

In [ ]:

viz.displayCollisions(False)
viz.displayVisuals(True)

In [ ]:
from time import sleep

In [ ]:
q_tr = np.loadtxt('panda_q_traj.txt')[:,:7]
v_tr = np.loadtxt('panda_q_traj.txt')[:,7:]
xyz_eff = end_eff(q_tr, tool_l=0.103)

In [ ]:
cart_v(q_tr[-1], v_tr[-1])

In [ ]:
viz.display(q_tr[0])
viz.viewer["sphere"].set_object(geo.Sphere(0.03),geo.MeshLambertMaterial(color=0x22dd22))
viz.viewer["sphere"].set_transform(tf.translation_matrix(xyz_eff[0,:]))

In [ ]:
def cart_v(q_p, qdot):
    jacmat = panda.jacob0(q = q_p, half = 'trans')
    return jacmat@ qdot

def grav_step(q, v, dt):
    q_n = q + dt * v
    v_n = v + dt * np.array([0,0,-9.8])
    return q_n, v_n

def grav_tray(q, v, dt, n_steps):
    q_arr = []
    for ii in range(n_steps):
        q_n, v_n = grav_step(q, v, dt)
        q_arr.append(q_n)
        q, v, = q_n, v_n
    return np.array(q_arr)

def brake_tray(q, v, dt, n_steps):
    if n_steps<1:
        return np.zeros([0,7])
    delta_t = dt*n_steps
    alpha = v/delta_t
    t_arr = np.expand_dims(np.linspace(0, delta_t, n_steps+1),1)
    q_arr = q + v*t_arr-0.5*alpha*t_arr**2
    return q_arr[1:]

In [ ]:
def display_robot_and_ball(robot_q, ball_q, frame_t = 0.04, t_end=1., n_extend = 25, brake_n = 10):
    
    dt = t_end/q_tr.shape[0]
    ball_v = cart_v(robot_q[-1], v_tr[-1])
    ball_add = grav_tray(ball_q[-1], ball_v, dt, n_extend)
    ball_q = np.concatenate([ball_q, ball_add], axis = 0)
    robot_add = brake_tray(robot_q[-1], v_tr[-1], dt, brake_n)
    robot_q = np.concatenate([robot_q, robot_add], axis=0)
    
    frames = max(robot_q.shape[0], ball_q.shape[0])
    for ii in range(frames):
        try:
            viz.display(robot_q[ii])
        except IndexError:
            viz.display(robot_q[-1])
        
        try:
            viz.viewer["sphere"].set_transform(tf.translation_matrix(ball_q[ii]))
        except IndexError:
            viz.viewer["sphere"].set_transform(tf.translation_matrix(ball_q[-1]))
        sleep(frame_t)

In [ ]:
for ii in range(75):
    viz.display(q_tr[ii])
    viz.viewer["sphere"].set_transform(tf.translation_matrix(xyz_eff[ii,:]))
    sleep(1/75)

In [ ]:
for jj in range(1):

In [ ]:
viz.viewer["traj"].set_object(
    geo.PointCloud(position=xyz_eff, color=np.random.rand(*xyz_eff.shape)),
)

In [ ]:
viz.viewer["traj"].set_object(geo.Line(geo.PointsGeometry(xyz_eff)))
viz.viewer["traj"].set_transform(tf.translation_matrix([0, 1, 0]))

In [ ]:
viz.viewer["box"].delete()

In [ ]:
viz.viewer["traj"].set_object(geo.PointCloud(position=xyz_eff, color=None))

In [ ]:
geo.Line(xyz_eff)